# Experiment of working with a model's log probabilities as forecasting probabilites

## Background
During pretraining, LLMs use the cross entropy loss, a proper scoring rule. Therefore they are trained to return their true best guess as to the next token. This is basically a forecasting task. Given what you have seen so far (in your context) and given what you have learned so far (in your weights from previous training examples), predict the next word. You have to build a world model and a model of language etc. to do so well. 

Now given that LLMs are pretrained this way, can't I elicit an LLMs true belief probability on a binary question, by asking it to return "yes" or "no" and looking at the log_probs for these tokens?

Apparently no. I just get the more likely result with almost certainty.

In [ ]:
import os
import re

import numpy as np
import openai
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
load_dotenv()

client = OpenAI(
    api_key = os.getenv("OPENAI_API_KEY"),
)

context_prompt = "Respond only with either yes or no by using the following machine readable format surrounding the response with two asteriks: **yes** or **no**"

In [ ]:
forecasting_question = "I'm throwing a six-sided die. Did it land on the number 1?"
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": f"{forecasting_question}"},
        {"role": "system", "content": f"{context_prompt}"},
    ],
    logprobs=True,
    top_logprobs=20,
    max_completion_tokens=5,
    temperature=2,
    # top_p=0.8,
    n=1,
)

In [ ]:
for i in range(2):
    token = response.choices[0].logprobs.content[1].top_logprobs[i].token
    prob = np.exp(response.choices[0].logprobs.content[1].top_logprobs[i].logprob)
    print(f"The probability of {token} is {prob}")

But somehow the model does argmax here instead of returning faithful probabilities. For anything with a probability lower than 0.5 it says no with almost certainty and for anything higher than a probability of 0.5 it says yes with almost certainty.

I tried gpt-4o, gpt-4o-mini, and gpt-3.5-turbo. With the probabilities getting closer to the extremes with more capable models.

Changing the temperature does not seem to influence this behaviour.


Example questions and responses:
```
forecasting_question = "I'm throwing a six-sided die. Did it land on a number lower than 5?"
```
 response:   
> The probability of yes is 0.999948788531352    
 The probability of unknown is 2.7535031259121803e-05


```
forecasting_question = "I'm throwing a six-sided die. Did it land on the number 3?"
```
response:   
>The probability of no is 0.9999919389784903  
The probability of yes is 7.88926171285407e-06 



And even when it should be 50/50 the response is often very strong in a random direction (or maybe leaning to no).
```
forecasting_question = "I'm throwing a fair coin. Will it show heads?"
```
response:  
>The probability of no is 0.8519258384620517  
The probability of yes is 0.1480425124768776  



